In [24]:
import numpy as np

class learning_rate_control():
    def __init__(self, max_step):
        self.learning_rate = np.zeros((max_step)) 
        self.learning_rate_current = 0.1
        
        self.triplet_loss = np.zeros((max_step))
        self.triplet_loss_smoothed = np.zeros((max_step))
        
        self.smooth_n = 3
        self.thresh = 0.05 # 5 percent
    def update_value(self, which_step, triplet_loss_value):
        self.triplet_loss[which_step] = triplet_loss_value
        # which_step is the index for the current value. start from 0.
        if which_step <= self.smooth_n - 1:
            num_existed_steps = which_step
            value_n_steps_agao = 0
            num_current_steps = num_existed_steps + 1
        else:
            num_existed_steps = self.smooth_n
            value_n_steps_agao = self.triplet_loss[which_step - self.smooth_n]
            num_current_steps = self.smooth_n
        if which_step == 0:
            self.triplet_loss_smoothed[which_step] = triplet_loss_value
            learning_rate_new = self.learning_rate_current
        else:
            self.triplet_loss_smoothed[which_step] = (self.triplet_loss_smoothed[which_step - 1] * num_existed_steps 
                                                      - value_n_steps_agao + triplet_loss_value)/num_current_steps
            learning_rate_new = self.learning_rate_change(which_step)

        ## update learning rate.
        self.learning_rate_current = learning_rate_new
        self.learning_rate[which_step] =  learning_rate_new
        return learning_rate_new
        
    def learning_rate_change(self, which_step):
        # whether the triplet_loss has been updated.
        if which_step >= 2 * self.smooth_n:
            smoothed_triplet_loss_curr = self.triplet_loss_smoothed[which_step]
            smoothed_triplet_loss_prev = self.triplet_loss_smoothed[which_step - self.smooth_n]
            
            if ((smoothed_triplet_loss_curr - smoothed_triplet_loss_prev)/smoothed_triplet_loss_prev) < self.thresh:
                return self.learning_rate_current * 0.5
            else:
                return self.learning_rate_current
        else:
            return self.learning_rate_current

In [25]:
a = learning_rate_control(10)

In [28]:
for ii in range(10):
    a.update_value(ii,  100 - 10 * ii)
    print(a.triplet_loss)
    print(a.triplet_loss_smoothed)

[ 100.    1.    2.    3.    4.    5.    6.    7.    8.    9.]
[ 100.     0.5    1.     2.     3.     4.     5.     6.     7.     8. ]
[ 100.   90.    2.    3.    4.    5.    6.    7.    8.    9.]
[ 100.   95.    1.    2.    3.    4.    5.    6.    7.    8.]
[ 100.   90.   80.    3.    4.    5.    6.    7.    8.    9.]
[ 100.   95.   90.    2.    3.    4.    5.    6.    7.    8.]
[ 100.   90.   80.   70.    4.    5.    6.    7.    8.    9.]
[ 100.   95.   90.   80.    3.    4.    5.    6.    7.    8.]
[ 100.   90.   80.   70.   60.    5.    6.    7.    8.    9.]
[ 100.   95.   90.   80.   70.    4.    5.    6.    7.    8.]
[ 100.   90.   80.   70.   60.   50.    6.    7.    8.    9.]
[ 100.   95.   90.   80.   70.   60.    5.    6.    7.    8.]
[ 100.   90.   80.   70.   60.   50.   40.    7.    8.    9.]
[ 100.   95.   90.   80.   70.   60.   50.    6.    7.    8.]
[ 100.   90.   80.   70.   60.   50.   40.   30.    8.    9.]
[ 100.   95.   90.   80.   70.   60.   50.   40.    7.    8.

In [30]:
5%3

2